<h2>Coursera Capstone Project Week 5</h2>

<h3>Battle of the Neighborhoods</h3>

<h4>Finding the best location for our restaurant business</h4>

Import all required libraries and packages

In [3]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         394 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-2.0.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5.

In [4]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Foursquare Credentials

In [5]:
CLIENT_ID = 'XGFSNYSXZIBZFE4EXYBPV3PUUT0OYZJPAOMC4E0YX5QHR0GU'
CLIENT_SECRET = 'U23MUAH41OSZIUJTWIZLOHVF5GPOE4BVM5ZRLGWP1DI05HCG'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XGFSNYSXZIBZFE4EXYBPV3PUUT0OYZJPAOMC4E0YX5QHR0GU
CLIENT_SECRET:U23MUAH41OSZIUJTWIZLOHVF5GPOE4BVM5ZRLGWP1DI05HCG


Data from Week 3

In [6]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset

In [7]:
import json

In [8]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)


In [9]:
neighborhoods_data = newyork_data['features']

In [10]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

Loop to fill our dataframe

In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    
neighborhoods.head(15)

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585
5,Bronx,Kingsbridge,40.881687,-73.902818
6,Manhattan,Marble Hill,40.876551,-73.910660
7,Bronx,Woodlawn,40.898273,-73.867315
8,Bronx,Norwood,40.877224,-73.879391
9,Bronx,Williamsbridge,40.881039,-73.857446


In [12]:
neighborhoods.shape

(306, 4)

Get coordinates for NYC

In [13]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The latitude coordinate of New York City is {}.'.format(latitude))
print('The longitude coordinate of New York City is {}.'.format(longitude))

The latitude coordinate of New York City is 40.7127281.
The longitude coordinate of New York City is -74.0060152.


Map of neighborhoods of NYC

In [40]:
nyc_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(nyc_map)  
    
nyc_map

Function to get venues

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=100,LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
nyc_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [20]:
print(nyc_venues.shape)
nyc_venues.head()

(797, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Kingsbridge,40.881687,-73.902818,Garden Gourmet Market,40.881350,-73.903389,Gourmet Shop
1,Kingsbridge,40.881687,-73.902818,MyUnique,40.881966,-73.903584,Thrift / Vintage Store
2,Kingsbridge,40.881687,-73.902818,Mattress Firm,40.881641,-73.903061,Mattress Store
3,Kingsbridge,40.881687,-73.902818,Stop & Shop,40.882048,-73.902111,Supermarket
4,Woodlawn,40.898273,-73.867315,Katonah Pizza and Pasta,40.898784,-73.867457,Pizza Place


In [21]:
nyc_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,3,3,3,3,3,3
Arlington,1,1,1,1,1,1
Astoria Heights,1,1,1,1,1,1
Bath Beach,3,3,3,3,3,3
Battery Park City,4,4,4,4,4,4
Bay Ridge,12,12,12,12,12,12
Bay Terrace,2,2,2,2,2,2
Bayswater,2,2,2,2,2,2
Bedford Park,1,1,1,1,1,1


In [22]:
len(nyc_venues['Venue Category'].unique())

214

In [23]:
nyc_onehot = pd.get_dummies(nyc_venues[['Venue Category']], prefix="", prefix_sep="")

In [24]:
nyc_onehot['Neighborhood'] = nyc_venues['Neighborhood']    

In [25]:
fixed_columns = [nyc_onehot.columns[-70]] + list(nyc_onehot.columns[:-70])

In [26]:
nyc_onehot = nyc_onehot[fixed_columns]

In [27]:
nyc_onehot.head()

,Opera House,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,High School,Himalayan Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Kingsbridge,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Kingsbridge,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,Kingsbridge,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Kingsbridge,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Woodlawn,0,0,0


In [28]:
nyc_onehot.shape

(797, 145)

Number of venues for each neghborhood

In [29]:
nyc_grouped = nyc_onehot.groupby('Neighborhood').mean().reset_index()
nyc_grouped

,Neighborhood,Opera House,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beach,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Café,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hawaiian Restaurant,Health Food Store,High School,Himalayan Restaurant,Historic Site,Home Service,Hookah Bar,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House
0,Allerton,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.333333,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000,0.00
1,Arlington,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.0,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.0,0.000000,0.000,0.000000,0.000000

Number of Donut shops in the neighborhood

In [30]:
nyc_grouped1 = nyc_grouped[["Neighborhood","Donut Shop"]]
nyc_grouped1

,Neighborhood,Donut Shop
0,Allerton,0.333333
1,Arlington,0.000000
2,Astoria Heights,0.000000
3,Bath Beach,0.000000
4,Battery Park City,0.000000
5,Bay Ridge,0.000000
6,Bay Terrace,0.000000
7,Bayswater,0.000000
8,Bedford Park,0.000000
9,Bensonhurst,0.000000


K Means for distribution into 5 clusters

In [31]:
klusters = 5
nyc_grouped_clustering = nyc_grouped1.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=klusters, random_state=0).fit(nyc_grouped_clustering)

kmeans.labels_[0:10]

array([3, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [32]:
nyc_merged = nyc_grouped1.copy()
nyc_merged["Cluster Labels"] = kmeans.labels_
nyc_merged.head()

,Neighborhood,Donut Shop,Cluster Labels
0,Allerton,0.333333,3
1,Arlington,0.000000,0
2,Astoria Heights,0.000000,0
3,Bath Beach,0.000000,0
4,Battery Park City,0.000000,0


In [33]:
nyc_merged = nyc_merged.join(neighborhoods.set_index('Neighborhood'), on='Neighborhood')
nyc_merged.head()

,Neighborhood,Donut Shop,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.333333,3,Bronx,40.865788,-73.859319
1,Arlington,0.000000,0,Staten Island,40.635325,-74.165104
2,Astoria Heights,0.000000,0,Queens,40.770317,-73.894680
3,Bath Beach,0.000000,0,Brooklyn,40.599519,-73.998752
4,Battery Park City,0.000000,0,Manhattan,40.711932,-74.016869


Cluster Map

In [34]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(klusters)
ys = [i + x + (i*x)**2 for i in range(klusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(nyc_merged['Latitude'], nyc_merged['Longitude'], nyc_merged['Neighborhood'], nyc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

df1 through 5 for our 5 clusters

In [36]:
df1 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 0]
df1

,Neighborhood,Donut Shop,Cluster Labels,Borough,Latitude,Longitude
1,Arlington,0.0,0,Staten Island,40.635325,-74.165104
2,Astoria Heights,0.0,0,Queens,40.770317,-73.894680
3,Bath Beach,0.0,0,Brooklyn,40.599519,-73.998752
4,Battery Park City,0.0,0,Manhattan,40.711932,-74.016869
5,Bay Ridge,0.0,0,Brooklyn,40.625801,-74.030621
6,Bay Terrace,0.0,0,Queens,40.782843,-73.776802
6,Bay Terrace,0.0,0,Staten Island,40.553988,-74.139166
7,Bayswater,0.0,0,Queens,40.611322,-73.765968
8,Bedford Park,0.0,0,Bronx,40.870185,-73.885512
9,Bensonhurst,0.0,0,Brooklyn,40.611009,-73.995180


In [37]:
df2 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 1]
df2

,Neighborhood,Donut Shop,Cluster Labels,Borough,Latitude,Longitude
10,Blissville,1.0,1,Queens,40.737251,-73.932442
100,Manor Heights,1.0,1,Staten Island,40.601810,-74.120594


In [38]:
df3 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 2]
df3

,Neighborhood,Donut Shop,Cluster Labels,Borough,Latitude,Longitude
44,East Harlem,0.142857,2,Manhattan,40.792249,-73.944182
83,Huguenot,0.142857,2,Staten Island,40.531912,-74.191741
96,Long Island City,0.105263,2,Queens,40.750217,-73.939202


In [39]:
df4 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 3]
df4

,Neighborhood,Donut Shop,Cluster Labels,Borough,Latitude,Longitude
0,Allerton,0.333333,3,Bronx,40.865788,-73.859319


In [42]:
df5 = nyc_merged.loc[nyc_merged['Cluster Labels'] == 4]
df5

,Neighborhood,Donut Shop,Cluster Labels,Borough,Latitude,Longitude
41,Downtown,0.033333,4,Brooklyn,40.690844,-73.983463
84,Hunters Point,0.041667,4,Queens,40.743414,-73.953868
85,Inwood,0.058824,4,Manhattan,40.867684,-73.921210


The cluster df1 and df2 have no or least competition and should be targeted first. Once a strong brand presence is achieved, we can further expand into neighborhoods belonging to clusters 3 to 5.